In [12]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

df = pd.read_csv("final_matches.csv")
# df = pd.read_csv("./EPL Standings 2000-2022.csv")

df


,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,team,season
0,2020-09-12,12:30,Premier League,Matchweek 1,Sat,Away,W,3,0,Fulham,...,Match Report,NaN,13.0,5.0,14.1,2.0,0,0,Arsenal,2021
1,2020-09-19,20:00,Premier League,Matchweek 2,Sat,Home,W,2,1,West Ham,...,Match Report,NaN,7.0,3.0,14.8,0.0,0,0,Arsenal,2021
2,2020-09-28,20:00,Premier League,Matchweek 3,Mon,Away,L,1,3,Liverpool,...,Match Report,NaN,4.0,3.0,15.0,0.0,0,0,Arsenal,2021
3,2020-10-04,14:00,Premier League,Matchweek 4,Sun,Home,W,2,1,Sheffield Utd,...,Match Report,NaN,6.0,5.0,16.3,0.0,0,0,Arsenal,2021
4,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Away,L,0,1,Manchester City,...,Match Report,NaN,11.0,3.0,18.4,2.0,0,0,Arsenal,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,2025-04-26,15:00,Premier League,Matchweek 34,Sat,Home,W,3,0,Leicester City,...,Match Report,NaN,20.0,5.0,19.8,1.0,0,0,Wolverhampton Wanderers,2025
3796,2025-05-02,20:00,Premier League,Matchweek 35,Fri,Away,L,0,1,Manchester City,...,Match Report,NaN,6.0,0.0,16.3,0.0,0,0,Wolverhampton Wanderers,2025
3797,2025-05-10,15:00,Premier League,Matchweek 36,Sat,Home,L,0,2,Brighton,...,Match Report,NaN,10.0,3.0,15.5,0.0,0,0,Wolverhampton Wanderers,2025
3798,2025-05-20,20:00,Premier League,Matchweek 37,Tue,Away,L,2,4,Crystal Palace,...,Match Report,NaN,12.0,3.0,15.1,0.0,0,0,Wolverhampton Wanderers,2025


### Test

### Fill null data

In [13]:
# # df[df.isnull().any(axis=1)][['homeXG', 'awayXG']]
# from sklearn.impute import SimpleImputer

# imputer = SimpleImputer(strategy='mean')
# df[['xg','xga', 'attendance']] = imputer.fit_transform(df[['xg','xga','attendance']])


### Encoder Referee

In [14]:
referee = sorted(df['referee'].unique())
dict_ref = {}

df['ref_encode'] = 0
# เก็บค่า ชื่อ กรรมการและค่า encode ลงไป
for i, v in enumerate(referee):
    dict_ref[v] = i
    index = df[df['referee'] == v].index
    df.loc[index, 'ref_encode'] = i
    
    
df[['referee', 'ref_encode']].sort_values(by='referee')



# Convert dict_ref to a DataFrame with columns 'referee' and 'code'
# df_ref = pd.DataFrame(list(dict_ref.items()), columns=['referee', 'code'])
# df_ref.to_csv('referee_encoding.csv', index=False)

,referee,ref_encode
2502,Andre Marriner,0
753,Andre Marriner,0
2825,Andre Marriner,0
2461,Andre Marriner,0
3472,Andre Marriner,0
...,...,...
1623,Tony Harrington,35
3664,Tony Harrington,35
1499,Tony Harrington,35
3238,Tony Harrington,35


### Prepare Data

In [15]:
# Cleaning
df['result_encode'] = df['result'].apply(lambda x: 0 if x == "L" else 1 if x == "W" else 2)

# Select features
features = ['xg', 'xga','ref_encode']
target = 'result'

# df_cleaned = df[df['venue'] == "Home"]

cleaned_df = df[df[target] != "D"]

cleaned_df[target]

0       W
1       W
2       L
3       W
4       L
       ..
3794    W
3795    W
3796    L
3797    L
3798    L
Name: result, Length: 2934, dtype: object

### Training Data

In [16]:

# Train-test split
X = cleaned_df[features]
y = cleaned_df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)


In [17]:
# Train Decision Tree Classifier
# model = MLPClassifier(hidden_layer_sizes=(20,20), activation='logistic', solver='adam', max_iter=1000, random_state=10)
model = DecisionTreeClassifier(criterion='entropy', max_depth=5, max_features=None, min_samples_leaf=1, min_samples_split=2, splitter='best', random_state=42)
model.fit(X_train, y_train)


,criterion,'entropy'
,splitter,'best'
,max_depth,5
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


### Evaluation

In [18]:
from sklearn.model_selection import cross_val_score, KFold

# ใช้ K-Fold Cross Validation (5-fold)
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# ประเมินความแม่นยำในแต่ละ fold
scores = cross_val_score(model, X, y, cv=kfold, scoring='accuracy')

print("📊 คะแนนแต่ละรอบ:", scores)
print("🎯 ความแม่นยำเฉลี่ย:", scores.mean())
print("🎯 ความแม่นยำ STD:", scores.std())

📊 คะแนนแต่ละรอบ: [0.76190476 0.74489796 0.7755102  0.81292517 0.77474403 0.7883959
 0.74744027 0.7883959  0.73720137 0.77133106]
🎯 ความแม่นยำเฉลี่ย: 0.7702746627661303
🎯 ความแม่นยำ STD: 0.022030913162612256


In [19]:
# Predict
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))

print(accuracy_score(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           L       0.77      0.82      0.79       591
           W       0.81      0.75      0.78       583

    accuracy                           0.79      1174
   macro avg       0.79      0.79      0.78      1174
weighted avg       0.79      0.79      0.78      1174

0.7853492333901193


### Chart of Decision Tree

## Prediction

In [20]:
df2 = pd.read_csv('../api/xG7.csv')
home = "Liverpool"
away = "Manchester Utd"

# get xg score by team
get_xg = lambda team: (df2[df2['team'] == team]['xg']).values[0]
get_xga = lambda team: (df2[df2['team'] == team]['xga']).values[0]

homeXG = get_xg(home)
homeXGA = get_xga(home)
awayXG = get_xga(away)
awayXGA = get_xga(away)

# homeXG = abs(homeXG - homeXGA)
# awayXG = abs(awayXG - awayXGA)

# label = ['xg','xga',]
data = [homeXG, awayXG, 30]

unseen_df = pd.DataFrame([data], columns=features)
y_pred = model.predict(unseen_df)
prob = model.predict_proba(unseen_df)
columns = model.classes_
probs_df = pd.DataFrame(prob, columns=columns)

probs_df.apply(lambda x: x.round(2) * 100)


,L,W
0,41.0,59.0


### Export Model

In [21]:
import pickle
f = open('../api/decisiontree.pkl', 'wb')
pickle.dump(model, f)
f.close()